In [4]:
#!/usr/bin/python3

from motion_control import Planner
import cv2
import visual_servo
import config
from mango_detection import mango_detector
import tensorflow as tf

In [5]:
camera_template = {
    'device' : cv2.VideoCapture('http://127.0.0.1:8081/stream.mjpg'),
    'focus_length' : config.camera_focus_length1,
    'offset' : [0, 0, 0], # x, y, z
}

tree_position = [1000, 3000, 5000]
base_position = [
    (500, 1),
    (1500, -1),
    (2500, 1),
    (3500, -1),
    (4500, 1),
    (5500, -1)
]
drop_position = {
    "Green": [0],
    "Yellow": [180],
}

planner = Planner.getInstance()
current_pos = [0, 0, 0]

mango_model = mango_detector.load_model('mango_detection/model/')

In [10]:
def get_distance(pos1, pos2):
    def diff(a1, a2):
        return np.abs(a1 - a2)
    return np.sqrt(diff(pos1[0], pos2[0])**2 + diff(pos1[1], pos2[1])**2 + diff(pos1[2], pos2[2])**2)

def get_workspace():
    workspace_x = 1000 / config.camera_focus_length3 * image_width * config.overlab_x
    workspace_y = 1000 / config.camera_focus_length3 * image_height * config.overlab_y
    return [workspace_x, workspace_y]

def pixel_to_distance(camera, pixel = [0, 0]): # x, y
    x = pixel
    

def next_position():
    global planner, current_pos
    current_pos = planner.get_pos()
    for idx, i in base_position:
        if np.abs(i[0] - current_pos) <= 100:
            if idx == len(base_position) - 1:
                planner.move_to_stereo_cam(0, 0, 0, config.default_speed)
            else:
                planner.move_to_stereo_cam(base_position[idx+1][0], current_pos[1], current_pos[2], config.default_speed)
            return

        
def state_wait_move(target):
    current_pos = planner.get_pos()
    return True if get_distance(current_pos, target) <= config.accept_move else False

def state_scan(image_height, image_width, max_length, i, j):
    workspace_x, workspace_y = get_workspace()
    x_pos = workspace_x / 2 + workspace_x * j
    y_pos = workspace_y / 2 + workspace_y * i
    
    target = [x_pos, y_pos, current_pos[2]]
    planner.move_to_stero_cam(target[0], target[1], target[2], config.default_speed)
    return state_wait_move(target)

def state_find_deg_maxscore(tf_session, camera):
    frame = camera['device'].read()
    if not frame[0]:
        print ("Camera Failed")
        #continue
        
    frame = frame[1]
    image_height, image_width, ch = frame.shape
    result = detect(frame, mango_model[1:], sess, 0.7)
    if result:
        center_image = [image_width/2, image_height/2] # x, y
        center_boxes = result['center'] # x, y
        
        center_diff = [center_boxes[0] - center_image[0], center_boxes[1] - center_image[1]] # x, y

def scan_mango():
    global cam_end_effector
    workspace_x = 1000 / config.camera_focus_length3 * image_width
    workspace_y = 1000 / config.camera_focus_length3 * image_height
    i_workspace = workspace_y * config.overlab_y
    j_workspace = workspace_x * config.overlab_x
    
    max_length = planer.get_max_length()
    count_j = max_length[0] / j_workspace
    count_i = max_length[1] / i_workspace
    
    current_pos = planner.get_pos()
    for i in range(1, count_i+1):
        y_pos = i_workspace / 2 + i * i_workspace
        for j in range(1, count_j+1):
            x_pos = j_workspace / 2 + j * j_workspace
            
            planner.move_to_stero_cam(x_pos, y_pos, current_pos[2], config.default_speed)
            while True:
                current_pos = planner.get_pos()
                if get_distance(current_pos, [x_pos, y_pos, current_pos[2]]) <= config.accept_move:
                    break
            
                
            

def drop_mango():
    pass

In [3]:
if __name__ == '__main__':
    detection_graph, tensor_dict, image_tensor = mango_model
    with detection_graph.as_default():
        with tf.Session() as sess:
    pass